In [1]:
import os
import datetime
import requests
import numpy as np
import pandas as pd
os.environ["CDF_LIB"] = f"C://Program Files/CDF_Distribution/cdf38_1-dist/lib"
from spacepy import pycdf

In [2]:
def data_processing(path):
    cdf = pycdf.CDF("Downloads" + path + ".cdf")
    df = pd.DataFrame({
        "BGSM1":[], 
        "BGSM2":[], 
        "BGSM3":[],
        "BGSE1":[],
        "BGSE2":[],
        "BGSE3":[],
        "Range":[],
        "Label":[]
    })
    
    startIndex = 0
    lst = [0, 0, 0, 0, 0, 0, 0]
    counter = 0
    
    for i in range(len(cdf['Epoch'][:])):
        if((cdf['Epoch'][i].item(0) - cdf['Epoch'][startIndex].item(0)).seconds < 1200):
            lst[0] += cdf['BGSE'][i][0]
            lst[1] += cdf['BGSE'][i][1]
            lst[2] += cdf['BGSE'][i][2]
            lst[3] += cdf['BGSM'][i][0]
            lst[4] += cdf['BGSM'][i][1]
            lst[5] += cdf['BGSM'][i][2]
            lst[6] += cdf['RANGE'][i]
            counter += 1       
        else:
            dic = {
                    "BGSM1":[lst[0].item(0)/counter], 
                    "BGSM2":[lst[1].item(0)/counter], 
                    "BGSM3":[lst[2].item(0)/counter],
                    "BGSE1":[lst[3].item(0)/counter],
                    "BGSE2":[lst[4].item(0)/counter],
                    "BGSE3":[lst[5].item(0)/counter],
                    "Range":[round(lst[6].item(0)/counter)],
                    "Label":0
            }
            df_dummy = pd.DataFrame(dic)
            df = pd.concat([df, df_dummy])
            startIndex = i
            lst = [0, 0, 0, 0, 0, 0, 0]
            counter = 0
    df = df.reset_index()
    df = df.drop(['index'], axis=1)
    df.to_csv("CSV/{path}.csv".format(path = path))

In [4]:
general = 'wi_h2_mfi_'
version = '05'
year = '2004'
month = '04'
        
for d in range(14, 16):
    if d < 10:
        day = '0' + str(d)
    else:
        day = str(d)
                    
    string = 'https://cdaweb.gsfc.nasa.gov/pub/data/wind/mfi/mfi_h2/{year}/wi_h2_mfi_{year}{month}{day}_v{version}.cdf'.format(
        year = year, month = month, day = day, version = version)
    response = requests.get(string)
    path = "/{general}{year}{month}{day}_v{version}".format(
        general = general, year = year, month = month, day = day, version = version)
    open("Downloads" + path + ".cdf", "wb").write(response.content)
    data_processing(path)
    print("{}{}{}".format(year, month, day))

20040414
20040415
